# Projet de reconnaissance faciale

# Reconnaissance faciale avec TensorFlow

## Preparation des données

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Charger le dataset
dataset_path = "dataset"
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Récupérer les noms de classes avant la transformation
class_names = train_dataset.class_names
print("Class names:", class_names)

# Normaliser les images
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Appliquer la normalisation
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 75 files belonging to 3 classes.
Using 60 files for training.
Found 75 files belonging to 3 classes.
Using 15 files for validation.
Class names: ['Edwin_Treny', 'Mathis_Wauters', 'Samuel_Jarjanette']


In [10]:
#import os
#import numpy as np
#import tensorflow as tf
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing import image
#
## Paramètres
#dataset_path = "dataset"
#batch_size = 32
#img_height = 224
#img_width = 224
#
## Récupérer les noms de classes
#class_names = os.listdir(dataset_path)
#class_names = [nom for nom in class_names if os.path.isdir(os.path.join(dataset_path, nom))]
#
## Charger manuellement les données
#images = []
#labels = []
#image_paths = []  # Nouvelle liste pour stocker les chemins des images
#
#for classe, nom in enumerate(class_names):
#    class_path = os.path.join(dataset_path, nom)
#    for img_file in os.listdir(class_path):
#        img_path = os.path.join(class_path, img_file)
#
#        # Charger et redimensionner l'image
#        img = image.load_img(img_path, target_size=(img_height, img_width))
#        img_array = image.img_to_array(img)
#
#        images.append(img_array)
#        labels.append(classe)
#        image_paths.append(img_path)  # Ajouter le chemin complet de l'image
#
## Convertir en numpy arrays
#images = np.array(images)
#labels = np.array(labels)
#image_paths = np.array(image_paths)
#
## Split stratifié
#X_train, X_val, y_train, y_val, paths_train, paths_val = train_test_split(
#    images, labels, image_paths,
#    test_size=0.2,
#    stratify=labels,
#    random_state=123
#)
#
## Normalisation
#normalization_layer = tf.keras.layers.Rescaling(1./255)
#
## Création des datasets TensorFlow
#train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
#val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
#
## Configuration des datasets
#train_dataset = (train_dataset
#    .batch(batch_size)
#    .map(lambda x, y: (normalization_layer(x), y))
#    .prefetch(tf.data.AUTOTUNE)
#)
#
#val_dataset = (val_dataset
#    .batch(batch_size)
#    .map(lambda x, y: (normalization_layer(x), y))
#    .prefetch(tf.data.AUTOTUNE)
#)
#
## Afficher les noms des fichiers de validation
#print("\nFichiers de validation :")
#for classe in range(len(class_names)):
#    print(f"\nClasse {class_names[classe]} :")
#    classe_val_paths = paths_val[y_val == classe]
#    for path in classe_val_paths:
#        print(os.path.basename(path))
#
## Autres informations de distribution
#print("\nNoms des classes:", class_names)
#print("Nombre total d'images:", len(images))
#print("Images d'entraînement:", len(X_train))
#print("Images de validation:", len(X_val))
#
## Vérification de la distribution des classes
#print("\nDistribution des classes totale:")
#unique, counts = np.unique(labels, return_counts=True)
#for classe, count in zip(unique, counts):
#    print(f"{class_names[classe]}: {count}")
#
#print("\nDistribution des classes dans l'entraînement:")
#unique_train, counts_train = np.unique(y_train, return_counts=True)
#for classe, count in zip(unique_train, counts_train):
#    print(f"{class_names[classe]}: {count}")
#
#print("\nDistribution des classes dans la validation:")
#unique_val, counts_val = np.unique(y_val, return_counts=True)
#for classe, count in zip(unique_val, counts_val):
#    print(f"{class_names[classe]}: {count}")

## Définir le modèle

In [11]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Charger MobileNetV2 sans la dernière couche
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,
                         weights='imagenet')

# Geler les couches du modèle pré-entraîné
base_model.trainable = False

# Ajouter des couches personnalisées
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')  # Utiliser class_names
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Entraîner le modèle

In [12]:
epochs = 10

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.3701 - loss: 1.3959 - val_accuracy: 1.0000 - val_loss: 0.3591
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step - accuracy: 0.7396 - loss: 0.6021 - val_accuracy: 0.9333 - val_loss: 0.2562
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 817ms/step - accuracy: 0.9007 - loss: 0.3560 - val_accuracy: 1.0000 - val_loss: 0.1213
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step - accuracy: 0.9458 - loss: 0.1371 - val_accuracy: 1.0000 - val_loss: 0.0656
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step - accuracy: 0.9778 - loss: 0.1236 - val_accuracy: 1.0000 - val_loss: 0.0471
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 785ms/step - accuracy: 0.9674 - loss: 0.1379 - val_accuracy: 1.0000 - val_loss: 0.0449
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step - accuracy: 0.9569 - loss: 0.0720 - val_accuracy: 1.0000 - val_loss: 0.0571
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step - accuracy: 0.9889 - loss: 0.0401 - val_accuracy: 1.0000 - val_loss: 0.

## Évaluer le modèle et ajuster les hyperparamètres

In [13]:
# Tester le modèle sur des données de validation
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation accuracy: {val_acc:.2f}")

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    ])

# Appliquer la data augmentation
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))

# Ré-entraîner le modèle avec la data augmentation
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

# Tester le modèle sur des données de validation
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation accuracy: {val_acc:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 1.0000 - loss: 0.0287
Validation accuracy: 1.00
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 930ms/step - accuracy: 0.8812 - loss: 0.3297 - val_accuracy: 1.0000 - val_loss: 0.0106
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 945ms/step - accuracy: 0.9028 - loss: 0.1771 - val_accuracy: 1.0000 - val_loss: 0.0313
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 917ms/step - accuracy: 0.9236 - loss: 0.1832 - val_accuracy: 0.9333 - val_loss: 0.0680
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.8806 - loss: 0.2654 - val_accuracy: 1.0000 - val_loss: 0.0322
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 888ms/step - accuracy: 0.9569 - loss: 0.0994 - val_accuracy: 1.0000 - val_loss: 0.0153
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 884ms/step - accuracy: 0.9569 - loss: 0.1540 - val_accuracy: 1.0000 - val_loss: 0.0109
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 903ms/step - accuracy: 0.9674 - loss: 0.0891 - val_accuracy: 1.0000 - val_loss: 0.0098
Epoch 8/10
2/2 ━━━━━━━

## Sauvegarder le modèle

In [ ]:
model.save("face_recognition_model.keras")


## Reconnaissance faciale en temps réel

In [7]:
import cv2
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array
import os
import tkinter as tk
from tkinter import Tk, Label
from tkinter import simpledialog
from tkinter import messagebox

# Charger le modèle sauvegardé
model = tf.keras.models.load_model("face_recognition_model.keras")

# Initialiser le classificateur de visage d'OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Dimensions des images attendues
img_height = 224
img_width = 224

# Seuil de confiance en dessous duquel la personne est considérée comme non reconnue
CONFIDENCE_THRESHOLD = 0.85

# Fonction pour effectuer une prédiction
def predict_face(face, model, class_names):
    face = cv2.resize(face, (img_width, img_height))  # Redimensionner
    face_array = img_to_array(face) / 255.0  # Normaliser
    face_array = np.expand_dims(face_array, axis=0)  # Ajouter une dimension batch
    predictions = model.predict(face_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_class, confidence

def show_message(message, duration=5):
    """
    Affiche un message dans une fenêtre pendant une durée spécifiée.
    """
    root = Tk()
    root.title("Prochaine étape")
    root.geometry("300x100")
    label = Label(root, text=message, font=("Helvetica", 16), wraplength=280)
    label.pack(expand=True)
    root.after(duration * 1000, root.destroy)  # Ferme la fenêtre après `duration` secondes
    root.mainloop()

def capture_face(prenom, nom):
    # Créer le dossier pour enregistrer les images si il n'existe pas
    folder_name = f"dataset/{prenom}_{nom}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Ouvrir la caméra
    cap = cv2.VideoCapture(0)
    captured_images = 0
    directions = ["Regardez a gauche", "Regardez a droite", "Regardez vers le haut", "Regardez vers le bas", "Regardez droit devant"]
    images_per_direction = 5  # Nombre d'images par direction
    total_images = len(directions) * images_per_direction

    direction_index = 0

    while captured_images < total_images:
        # Afficher le texte de l'étape actuelle
        current_direction = directions[direction_index]
        print(f"Etape : {current_direction}")

        # Pause de 3 secondes avec affichage de l'étape actuelle
        for i in range(3, 0, -1):
            ret, frame = cap.read()
            if not ret:
                break
            # Ajouter le texte pour guider l'utilisateur
            cv2.putText(frame, f"{current_direction} dans {i} secondes",
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.imshow('Capturer le visage', frame)
            cv2.waitKey(1000)

        # Capturer les 50 images pour cette étape
        while captured_images < (direction_index + 1) * images_per_direction:
            ret, frame = cap.read()
            if not ret:
                break

            # Convertir en niveaux de gris pour la détection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Détection des visages
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                # Extraire le visage
                face = frame[y:y+h, x:x+w]

                # Enregistrer l'image
                face_filename = f"{folder_name}/face_{captured_images}.jpg"
                cv2.imwrite(face_filename, face)
                captured_images += 1

                # Ajouter un rectangle et des informations à l'écran
                cv2.putText(frame, f"Image {captured_images}/{total_images}",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Afficher l'étape actuelle pendant la capture
            cv2.putText(frame, f"Etape actuelle : {current_direction}",
                        (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            # Afficher le flux vidéo avec les visages détectés
            cv2.imshow('Capturer le visage', frame)

            # Quitter avec la touche 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return

        # Passer à la prochaine direction
        direction_index = (direction_index + 1) % len(directions)

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()
    show_message(f"Enregistrement terminé. Les visages ont été enregistrés dans le dossier : {folder_name}")

# Fonction pour analyser les visages
def analyze_face():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir en niveaux de gris pour la détection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Détection des visages
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Extraire le visage
            face = frame[y:y+h, x:x+w]

            # Prédire le nom
            predicted_class, confidence = predict_face(face, model, class_names)

            # Si la confiance est inférieure au seuil, afficher "Inconnu"
            if confidence < CONFIDENCE_THRESHOLD:
                color = (0, 0, 255)  # Rouge pour le rectangle
                predicted_class = "Inconnu"
                text = f"{predicted_class} ({confidence*100:.1f}%)"
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

            # Dessiner un rectangle autour du visage et ajouter le texte
            color = (0, 255, 0)  # Vert pour le rectangle
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            text = f"{predicted_class} ({confidence*100:.1f}%)"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Afficher le flux vidéo avec les prédictions
        cv2.imshow('Reconnaissance Faciale', frame)

        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()

# Interface graphique avec Tkinter
def create_interface():
    # Fenêtre principale
    window = tk.Tk()
    window.title("Reconnaissance Faciale")

    # Fonction pour enregistrer un visage
    def start_capture():
        prenom = simpledialog.askstring("Prénom", "Entrez votre prénom:")
        nom = simpledialog.askstring("Nom", "Entrez votre nom:")
        if prenom and nom:
            capture_face(prenom, nom)

    # Fonction pour analyser les visages
    def start_analyze():
        analyze_face()

    # Boutons pour chaque fonctionnalité
    capture_button = tk.Button(window, text="Enregistrer un visage", command=start_capture)
    capture_button.pack(pady=10)

    analyze_button = tk.Button(window, text="Analyser un visage", command=start_analyze)
    analyze_button.pack(pady=10)

    # Lancer l'interface
    window.mainloop()

# Lancer l'interface graphique
create_interface()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━